In [24]:
from fbchat import Client
from fbchat.models import *
from fbchat import log
from random import randint
from pword import password
from collections import Counter

version = 3

username = "robbot8947"

staging = False
if staging:
    thread_id = "5820049228023910"
else:
    thread_id = "4854566911328063"

# Subclass fbchat.Client and override required methods
class Heardler:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.scores = {}
    
    def average_score(self):
        if len(self.scores.values()) > 0:
            return sum(self.scores.values()) / len(self.scores.values())
        else:
            return "no scores for player"

    def minimum_score(self):
        if len(self.scores.values()) > 0:
            return min(self.scores.values())
        else:
            return "no scores for player"

    def maximum_score(self):
        if len(self.scores.values()) > 0:
            return max(self.scores.values())
        else:
            return "no scores for player"

    def leaderboard_message(self):
        # returns a string with the leaderboard message showing score distribution for the player
        if len(self.scores.values()) > 0:
            score_list = self.scores.values()
            individual_scores = Counter(score_list)
            print_str = f"PLAYER {self.name.upper()} SCOREBOARD:\n"
            total = sum(individual_scores.values())
            print(f"total = {total}")
            possible_scores_list = [1,2,3,4,5,6]
            for score in possible_scores_list:
                print(f"score: {score}")
                try:
                    occurences = individual_scores[score]
                except:
                    occurences = 0
                print(f"occurences: {occurences}")

                multiplier = round(15*(occurences)/(total))

                print_str += f"{score}: "
                if occurences:
                    print_str += "■"*multiplier + (15-multiplier)*" " + f"({occurences})" + "\n"
                else:
                    print_str += "\n"
            
            return print_str
        else:
            return "no scores for player"

        

class HeardleBot(Client):
    awake = True
    def onMessage(self, author_id, message_object, thread_id, thread_type, **kwargs):
        self.markAsDelivered(thread_id, message_object.uid)
        self.markAsRead(thread_id)
        self.process_message(author_id, message_object, thread_id, thread_type, False)

    def process_message(self, author_id, message_object, thread_id, thread_type, bootup_read=True):
        # add this check so that rob does not reply to commands he lists in his own patch notes
        if (author_id != self.uid) and (message.text != None) and message_object.author in user_dict.keys():
            # log.info("{} from {} in {}".format(message_object, thread_id, thread_type.name))

            protein = (message_object.text).lower() # cast message to lowercase so that all command checks aren't case sensitive
            auth = message_object.author
            message_author = user_dict[auth]
            uname = message_author.name
            first_name = uname.split(' ')[0]

            # print(protein)

            loser = False

            send_message = False

            # initial check if the message that has been sent is a heardle message
            if "#heardle" in protein:
                valid_score = False
                print(f"message: {protein}")
                heardle_number = self.number_from_message(protein)
                if heardle_number == None:
                    # leave function, invalid entry
                    return
                # calculate score. golf scoring
                score = (protein.count('⬛️') + protein.count('🟥') + protein.count('🟩'))

                # check score is valid and that score has not already been submitted for today
                if heardle_number not in message_author.scores.keys():
                    if score <= 6 and score >= 0:
                        # i.e. valid score
                        valid_score = True
                        message_author.scores[heardle_number] = score
                        message_string = (f"{first_name} scored {score} for Heardle #{heardle_number}").upper()
                        send_message = True
                    else: # score is not valid
                        if score > 6:
                            message_string = (f"{first_name} probably cheated on Heardle #{heardle_number} today. score {score} is impossible, maximum score is 6").upper()
                            send_message = True
                        elif score < 0 :
                            message_string = (f"{first_name} probably didnt score {score} for Heardle #{heardle_number}, because score can't be negative").upper()
                            send_message = True
                else:
                    message_string = (f"{first_name} has already submitted a score for Heardle #{heardle_number}").upper()
                    send_message = True

                # check if player is a loser or not and score zeros                
                if score == 0:
                    loser = True         
        
                # set message to message decided in previous block, but transformed to uppercase
                message_object.text = message_string.upper()
                send_message = True

                # send message set before unless it is the initial bootup read, in which case we 
                # don't want to spam the chat with messages
                if send_message and not bootup_read:
                    self.send(message_object, thread_id=thread_id, thread_type=thread_type)
                    if not valid_score:
                        message_object.text = 'SOMETHING PROBABLY WENT WRONG, SCORE NOT RECORDED. MAYBE TRY AGAIN'
                        self.send(message_object, thread_id=thread_id, thread_type=thread_type)
                    # i.e. if score is low
                    elif loser:
                        message_object.text = insults[randint(0, len(insults)-1)]
                        loser = False
                        self.send(message_object, thread_id=thread_id, thread_type=thread_type)

            # logic tree for commands
            # if message isn't a heardle message, check to see if there is a command for rob in the message
            elif '/rob' in protein and not bootup_read:
                
                # first command /rob here, reports whether rob is here and if he is asleep or not
                if 'status' in protein:
                    send_message = True
                    if self.awake:
                        message_object.text = 'RUNNING'
                    else:
                        message_object.text = 'ASLEEP'

                # second command /rob wake, will wake rob up if he is asleep
                elif 'wake' in protein:
                    self.awake = True
                    message_object.text = "AWAKE"
                    send_message = True

                # third command, /rob sleep, puts rob to sleep. rob will not 
                elif 'sleep' in protein:
                        message_object.text = "ZZZ"
                        self.awake = False
                        send_message = True
                
                # these commands depend on rob being awake
                elif self.awake:
                    # fourth command, /rob notes, rob will send patch notes for current version ot the chat
                    if 'notes' in protein:
                        message_object.text = notes
                        send_message = True
                    
                    # if none have triggered  
                    else:
                        message_object.text = 'NO COMMAND DETECTED\n' + commands
                        send_message = True
                else:
                    send_message = False

                if send_message:
                    self.send(message_object, thread_id=thread_id, thread_type=thread_type)

    def number_from_message(self, message):
        start_index = 10
        num_str = ""
        for char in message[start_index:]:
            if char.isnumeric():
                num_str += char
                # print(char)
                # print(num_str)
            else:
                break
        try:
            heardle_number = int(num_str)
        except(ValueError):
            heardle_number = None
        return heardle_number

def leaderboard(users):
    # returns a string giving a leaderboard of all chat members ranked by average score
    scores = []
    for player in users.values():
        score = player.average_score()
        if type(score) == int:
            scores.append((player.name, score))
        else:
            scores.append((player.name, 0))
    sorted_by_score = sorted(scores, key=lambda tup: tup[1])
    print_str = "LEADERBOARD: \n"
    i = 1
    for name, score in sorted_by_score:
        print_str += f"{i}." + name.upper() + str(score) + "\n"
        i += 1
        
    return(print_str)



insults = ["and they call me a bot".upper()]
wake_message = f"ROB_BOT V{version} BOOTING"
boot_complete = "BOOT COMPLETE"
commands = 'valid commands are: status, wake, sleep, notes'.upper()

client = HeardleBot(username, password)

thread_type = ThreadType.GROUP

# open patch notes file for corresponding version and read them
file = open(f'patch_notes/patch_notes{version}.txt')
notes = file.read()
file.close()

# print(notes)

print('fetching threads:')
threads = client.fetchThreadList()
# print(threads)

for thread in threads:
    # print(thread)
    if thread.uid == thread_id:
        group = thread
        print('thread found')
    print('\n')

user_dict = {}

for user_id in group.participants:
    # loop through users in thread and create a heardler object for each
    user = client.fetchUserInfo(user_id)[user_id]
    print(user)
    print("user's name: {}".format(user.name))
    
    user_dict[user_id] = Heardler(user_id, user.name)

# print(user_dict)

Logging in robbot8947...
Login of robbot8947 successful.


URL https://m.facebook.com/?_rdr


fb_dtsg_element:

<input autocomplete="off" name="fb_dtsg" type="hidden" value="AQEqLvYBBeDWoxw:21:1648119444"/>


fb_dtsg:

AQEqLvYBBeDWoxw:21:1648119444
revision 1
fetching threads:
thread found








User(uid='100079006569868', type=ThreadType.USER, photo='https://scontent.fchc1-1.fna.fbcdn.net/v/t1.30497-1/143086968_2856368904622192_1959732218791162458_n.png?stp=cp0_dst-png_p32x32&_nc_cat=1&ccb=1-5&_nc_sid=0081f9&_nc_ohc=0p3LWDxlPqcAX-RyGM4&_nc_ht=scontent.fchc1-1.fna&oh=00_AT85z00oyjVAaOAzChoCQmByQgxnNvwXrmS8O1ZmD2r_pw&oe=6263A2F8', name='Rob Bot', last_message_timestamp=None, message_count=None, plan=None, url='https://www.facebook.com/robbot8947', first_name='Rob', last_name=None, is_friend=False, gender='neuter_singular', affinity=None, nickname=None, own_nickname=None, color=None, emoji=None)
user's name: Rob Bot
User(uid='100004858381029', type=ThreadType.USER, photo='https://scontent.fchc1-1.fna.fbcdn.net/v/t39.30808-1/272049688_328874363

In [25]:
# fetch messages to the thread
messages = client.fetchThreadMessages(thread_id, limit = 10000)
# Since the message come in reversed order, reverse them
messages.reverse()

print(f"messages retrieved: {len(messages)}")
for message in messages:
    client.process_message(message.uid, message, thread_id, thread_type)
    #print('processed message successfully')



messages retrieved: 345
message: #heardle #11

🔉⬛️⬛️🟩⬜️⬜️⬜️

https://heardle.app
message: #heardle #12

🔊🟩⬜️⬜️⬜️⬜️⬜️
message: #heardle #12

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #13

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #13

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #14

🔈⬛️⬛️⬛️⬛️⬛️🟩

https://heardle.app
message: #heardle #14

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #15

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #15

🔈⬛️⬛️⬛️⬛️⬛️🟩

https://heardle.app
message: #heardle #16

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #16

🔇⬛️⬛️⬛️⬛️⬛️⬛️

https://heardle.app
message: #heardle #17

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #17

🔊🟩⬜️⬜️⬜️⬜️⬜️

https://heardle.app
message: #heardle #17

🔉⬛️🟥🟩⬜️⬜️⬜️

https://heardle.app
message: #heardle #18

🔈⬛️⬛️⬛️⬛️🟩⬜️

https://heardle.app
message: #heardle #18

🔈⬛️⬛️⬛️⬛️🟩⬜️

https://heardle.app
message: #heardle #18

🔇⬛️⬛️⬛️⬛️⬛️⬛️

https://heardle.app
message: #heardle #19

🔊🟩⬜️

In [26]:
for key in user_dict:
    player = user_dict[key]
    print(player.name)
    for score_key in player.scores:
        print(f"heardle no. {score_key}, score = {player.scores[score_key]}")
    print(f"average score: {player.average_score()}")
    print(f"best score: {player.minimum_score()}")
    print(f"worst score: {player.maximum_score()}")
    print(player.leaderboard_message())

Rob Bot
average score: no scores for player
best score: no scores for player
worst score: no scores for player
no scores for player
Eli Briggs O'Neill
heardle no. 11, score = 3
heardle no. 12, score = 1
heardle no. 13, score = 1
heardle no. 14, score = 6
heardle no. 17, score = 3
heardle no. 18, score = 5
heardle no. 19, score = 2
heardle no. 20, score = 5
heardle no. 21, score = 5
heardle no. 22, score = 4
heardle no. 23, score = 3
heardle no. 24, score = 6
heardle no. 25, score = 6
heardle no. 26, score = 1
heardle no. 27, score = 5
average score: 3.7333333333333334
best score: 1
worst score: 6
total = 15
score: 1
occurences: 3
score: 2
occurences: 1
score: 3
occurences: 3
score: 4
occurences: 1
score: 5
occurences: 4
score: 6
occurences: 3
PLAYER ELI BRIGGS O'NEILL SCOREBOARD:
1: ■■■            (3)
2: ■              (1)
3: ■■■            (3)
4: ■              (1)
5: ■■■■           (4)
6: ■■■            (3)

Teneale Guy
heardle no. 20, score = 1
heardle no. 23, score = 3
heardle no. 

In [27]:
print(leaderboard(user_dict))

None
